## StationaryPFR

Plug flow reactor in steady state operation.



In [ ]:
import matplotlib.pyplot as plt

import reactord as rd

### Operating the PFR

### Setting the kinetic laws

### Thermal operations

#### Isothermic

#### Adiabatic

#### No-isothermic